In [1]:
import os
import torch
import GPUtil
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from model.model import PODNonlinearModel
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [2]:
import matplotlib.pyplot as plt
from vecopsciml.operators.zero_order import Mx, My
import time

In [3]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/non_linear/non_linear_10000.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/non_linear')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/non_linear/POD_model')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear/POD_model


In [4]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear/non_linear_10000.pkl


In [5]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [6]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


In [7]:
# Train data splitting in train/test
X = torch.tensor(dataset['X_train'], dtype=torch.float32).unsqueeze(1)
y = torch.tensor(dataset['y_train'], dtype=torch.float32).unsqueeze(1)
K = torch.tensor(dataset['k_train'], dtype=torch.float32).unsqueeze(1)
f = torch.tensor(dataset['f_train'], dtype=torch.float32).unsqueeze(1)

X_train, X_test, y_train, y_test, K_train, K_test, f_train, f_test = train_test_split(X, y, K, f, test_size=0.3, random_state=42)

# Data processing and adequacy with our TensOps library
X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)

y_train = TensOps(y_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_test = TensOps(K_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

f_train = TensOps(f_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_test = TensOps(f_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

# Loading and processing validation data
X_val = torch.tensor(dataset['X_val'], dtype=torch.float32).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

In [8]:
U_train, S_train, Vt_train = torch.linalg.svd(y_train.values.detach().squeeze().to('cpu').view(y_train.values.detach().shape[0], -1).T, full_matrices=False)

error = []
for mode_i in range(len(S_train)):
    error.append((sum(S_train[:mode_i])/sum(S_train)).numpy())
    if mode_i < 20:
        print(mode_i, '-->', error[mode_i])

0 --> 0.0
1 --> 0.90155274
2 --> 0.9484499
3 --> 0.99119854
4 --> 0.99418914
5 --> 0.99641645
6 --> 0.99841464
7 --> 0.99887806
8 --> 0.9992371
9 --> 0.99948627
10 --> 0.99969864
11 --> 0.99978
12 --> 0.9998439
13 --> 0.9998923
14 --> 0.9999196
15 --> 0.99994326
16 --> 0.99995667
17 --> 0.9999685
18 --> 0.999976
19 --> 0.9999819


In [9]:
num_modes = 13

start_time = time.time()

U_train, S_train, Vt_train = torch.linalg.svd(y_train.values.detach().squeeze().to('cpu').view(y_train.values.detach().shape[0], -1).T, full_matrices=False)

U_reduced_train = U_train[:, :num_modes]
S_reduced_train = S_train[:num_modes]
Vt_reduced_train = Vt_train[:num_modes, :]

POD_base = torch.mm(U_reduced_train, torch.diag(S_reduced_train)).to(DEVICE)
# y_train = Vt_reduced_train.T

end_time = time.time()

print(f"Tiempo de ejecución: {end_time - start_time:.6f} segundos")

Tiempo de ejecución: 0.026903 segundos


In [10]:
# Predictive network architecture
input_shape = X_train[0].shape
predictive_layers = [20, 10, num_modes]
predictive_output = y_train.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train)).values[0].shape

# Other parameters
n_filters_explanatory = 5

In [11]:
# Load model and the optimizer
model = PODNonlinearModel(input_shape, predictive_layers, POD_base, predictive_output, explanatory_input, explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 10000

batch_size = 64
n_checkpoints = 10

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D, n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 1.650e+07, Test loss: 2.493e+07, MSE(e): 1.595e+00, MSE(pi1): 5.137e+01, MSE(pi2): 6.652e-01, MSE(pi3): 3.288e-01
Epoch 100, Train loss: 1.141e+05, Test loss: 2.258e+05, MSE(e): 1.087e-02, MSE(pi1): 1.043e-01, MSE(pi2): 7.283e-03, MSE(pi3): 4.391e-02
Epoch 200, Train loss: 4.961e+05, Test loss: 1.154e+06, MSE(e): 4.953e-02, MSE(pi1): 3.282e-02, MSE(pi2): 2.153e-02, MSE(pi3): 4.627e-03
Epoch 300, Train loss: 6.206e+04, Test loss: 1.427e+05, MSE(e): 6.156e-03, MSE(pi1): 1.876e-02, MSE(pi2): 3.359e-03, MSE(pi3): 3.044e-03
Epoch 400, Train loss: 1.524e+04, Test loss: 4.292e+04, MSE(e): 1.502e-03, MSE(pi1): 1.177e-02, MSE(pi2): 1.369e-03, MSE(pi3): 1.004e-03
Epoch 500, Train loss: 2.512e+04, Test loss: 1.032e+05, MSE(e): 2.488e-03, MSE(pi1): 1.356e-02, MSE(pi2): 1.898e-03, MSE(pi3): 1.093e-03
Epoch 600, Train loss: 4.615e+04, Test loss: 4.962e+04, MSE(e): 4.601e-03, MSE(pi1): 6.073e-03, MSE(pi2): 2.343e-03, MSE(pi3): 8.314e-04
Epoch 700, 

In [12]:
# Parametros de entrenamiento
start_epoch = 9000
n_epochs = 100000

batch_size = 64
n_checkpoints = 100

second_lr = 1e-4

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 9000.
Epoch 9000, Train loss: 1.118e+04, Test loss: 2.776e+04, MSE(e): 1.103e-03, MSE(pi1): 5.658e-03, MSE(pi2): 7.166e-04, MSE(pi3): 9.157e-04
Epoch 9100, Train loss: 9.193e+03, Test loss: 1.818e+04, MSE(e): 9.094e-04, MSE(pi1): 3.376e-03, MSE(pi2): 5.771e-04, MSE(pi3): 6.466e-04
Epoch 9200, Train loss: 8.583e+03, Test loss: 1.710e+04, MSE(e): 8.490e-04, MSE(pi1): 3.020e-03, MSE(pi2): 5.467e-04, MSE(pi3): 6.187e-04
Epoch 9300, Train loss: 8.632e+03, Test loss: 1.537e+04, MSE(e): 8.542e-04, MSE(pi1): 2.918e-03, MSE(pi2): 5.486e-04, MSE(pi3): 5.977e-04
Epoch 9400, Train loss: 7.964e+03, Test loss: 1.440e+04, MSE(e): 7.876e-04, MSE(pi1): 3.019e-03, MSE(pi2): 5.199e-04, MSE(pi3): 5.681e-04
Epoch 9500, Train loss: 7.550e+03, Test loss: 1.372e+04, MSE(e): 7.463e-04, MSE(pi1): 3.047e-03, MSE(pi2): 5.018e-04, MSE(pi3): 5.526e-04
Epoch 9600, Train loss: 7.203e+03, Test loss: 1.316e+04, MSE(e): 7.117e-04, MSE(pi1): 3.061e-03, MSE(pi2): 4.861e-04, MSE(p